In [1]:
from datetime import datetime
import pandas as pd
from pathlib import Path
import pickle

In [5]:
csvs = [x for x in Path('data').iterdir() if 'timeseries' in str(x)]
counts = []
regions = []
results = pd.DataFrame(columns=['roi', 'new_deaths', 'new_cases', 'new_recover', 'abs_days', 'rel_days'])
i = 1
for csv in csvs:
    region = str(csv).split('.')[0].split('_')[-1]
    regions.append(region)
    df = pd.read_csv(csv)
    df = df.iloc[:-1]
    df = df[df['cum_cases']>=10]
    df['abs_days'] = df['dates2'].apply(lambda x: (datetime.strptime(x, '%m/%d/%y') - datetime.strptime('01/22/20', '%m/%d/%y')).days)
    df['rel_days'] = df['dates2'].apply(lambda x: (datetime.strptime(x, '%m/%d/%y') - datetime.strptime(df['dates2'].values[0], '%m/%d/%y')).days)
    counts.append(df.shape[0])
    subset = ['new_deaths', 'new_cases', 'new_recover']
    df[subset] = df[subset].clip(lower = 0)
    df['roi'] = i
    subset = ['roi', 'new_deaths', 'new_cases', 'new_recover', 'abs_days', 'rel_days']
    results = results.append(df[subset])
    i += 1

In [6]:
import pickle
with open('data/stacked.pkl', 'wb') as f:
    pickle.dump({'data': results, 'regions': regions, 'counts': counts}, f)

In [7]:
results

,roi,new_deaths,new_cases,new_recover,abs_days,rel_days
9,1,0,1,0,54,0
10,1,0,7,0,55,1
11,1,0,12,0,56,2
12,1,1,15,0,57,3
13,1,0,5,0,58,4
...,...,...,...,...,...,...
30,39,61,1143,0,73,22
31,39,77,1172,0,74,23
32,39,110,1283,0,75,24
33,39,118,973,0,76,25
